# Problem 1 - Working with RDDs (5 points)

This is an interactive PySpark session. Remember that when you open this notebook the `SparkContext` and `SparkSession` are already created, and they are in the `sc` and `spark` variables, respectively. You can run the following two cells to make sure that the Kernel is active.

**Do not insert any additional cells than the ones that are provided.**

In [ ]:
sc

In [ ]:
spark

In the following cell, make an RDD called `top1m` that contains the contents of the file `top-1m.csv` that you placed into the cluster's HDFS.

In [ ]:
top1m=spark.read.csv("hdfs://user/hadoop/top-1m.csv",header=False).withColumnRenamed('Rank', 'Site')

In [ ]:
top1m=sc.textFile("hdfs://user/hadoop/top-1m.csv").withColumnRenamed('Rank', 'Site')

There is one element in the RDD for each line in the file. The `.count()` method will compute how many lines are in the file. In the following cell, type the expression to count the lines in the `top1m` RDD. Run the cell and see the result.

In [ ]:
top1m.count()

## Count the `.com` domains

How many of the websites in this RDD are in the .com domain?

In the following cell, write a code snippet that finds the records with `.com` and counts them. (Hint: use a regular expression.)

In [ ]:
top1m.select(top1m.Site.endswith('com').alias('endswithCom')).filter(lambda x: x=True).count()

## Histogram the Top Level Domains (TLDs)

What is the distribution of TLDs in the top 1 million websites? We can quickly compute this using the RDD function `countByValue()`.

In the following cell, write a function called `tld` (in Python) that takes a domain name string and outputs the top-level domain.

In [ ]:
import re

def tld(i):
    domain= re.compile(r'[a-z]{2,3}').findall(i.lower())[-1]
    return(domain)

In the following cell, map the `top1m` RDD using `tld` into a new RDD called `tlds`. 

In [ ]:
tlds=top1m.map(domain)

In the following two cells, evaluate `top1m.first()` and  `tlds.first()` to see if the first line of `top1m` transformed by `tld` is properly represented as the first line of `tlds`. 

In [ ]:
top1m.first()

In [ ]:
tlds.first()

Look at the first 50 elements of `top1m` by evaluating `top1m.take(50)`.

In [ ]:
top1m.take(50)

Try the same thing with the `tlds` RDD to make sure that the first 50 lines were properly transformed.


In [ ]:
tlds.take(50)

At this point, `tlds.countByValue()` would give us a list of each TLD and the number of times that it appears in the top1m file. Note that this function returns the results as a `defaultDict` in the Python environemnt, not as an RDD. But we want it reverse sorted by count. To do this, we can set a variable called `tlds_and_counts` equal to `tlds.countByValue()` and then reverse the order, sort, and take the top 50, like this:

```
tlds_and_counts = tlds.countByValue()
counts_and_tlds = [(count,domain) for (domain,count) in tlds_and_counts.items()]
```

In the following cell, run the code above to produce the Python Dictionary.

In [ ]:
tlds_and_counts = tlds.countByValue()
counts_and_tlds = [(count,domain) for (domain,count) in tlds_and_counts.items()]

In the following cell, reverse sort `counts_and_tlds` and display the first 50.

In [ ]:
sorted(counts_and_tlds.items(), key=lambda x: x[1]).reverse().head(50)

**Question:** `top1m.collect()[0:50]` and `top1m.take(50)` produce the same result. Which one is more efficient and why? Put your answer in the cell below.

In [ ]:
## Answer Here (don't run this cell)
The latter one. Because the function of collect will load all of the elements of the RDD, which will spend a lot of time and memory.

When you finish this problem, click on the File -> 'Save and Checkpoint' in the menu bar to make sure that the latest version of the workbook file is saved. Also, before you close this notebook and move on, make sure you disconnect your SparkContext, otherwise you will not be able to re-allocate resources. Remember, you will commit the .ipynb file to the repository for submission (in the master node terminal.)

In [ ]:
sc.stop()